<a href="https://colab.research.google.com/github/hkaido0718/IncompleteDiscreteChoice/blob/main/IdentifiedSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inequalities $\to$ Identified Set

We saw how to derive the sharp identifying restrictions (inequalities) from a graph. Suppose $P(\cdot|x)$ is known. The _sharp identified set_ $\Theta_I(P)$ is the set of $\theta$ values that are compatible with $P$ and all model restrictions. That is,
$$\Theta_I(P)\equiv\{\theta:P(A|x)\ge \nu_\theta(A|x),~x\in\mathcal X, A\subseteq \mathcal Y\}.$$

Let's try to obtain this set. First, we load the `idc` library.

In [1]:
!git clone https://github.com/hkaido0718/IncompleteDiscreteChoice.git

Cloning into 'IncompleteDiscreteChoice'...
remote: Enumerating objects: 287, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 287 (delta 84), reused 49 (delta 28), pack-reused 144
Receiving objects: 100% (287/287), 820.16 KiB | 4.25 MiB/s, done.
Resolving deltas: 100% (152/152), done.


# Example: Entry game

We used $(F_\theta(a),F_\theta(b),F_\theta(c),F_\theta(d),F_\theta(e))=(0.1, 0.2, 0.3, 0.15, 0.25)$, which was arbitrary. Let's derive the probability allocation from a parametric model.

Earlier, we considered the following regions
\begin{align*}
\text{region00} &= \{U_1 \leq -x_1\beta_1, U_2 \leq -x_2\beta_2\} \\
\text{region01} &= \{U_1 \leq -x_1\beta_1 - \delta_1,U_2 \geq -x_2\beta_2\} \\
\text{region10} &= \{U_1 \geq -x_1\beta_1, U_2 \leq -x_2\beta_2 - \delta_2\} \\
\text{region11} &= \{U_1 \geq -x_1\beta_1 - \delta_1, U_2 \geq -x_2\beta_2 - \delta_2\}
\end{align*}



Suppose $(U_1,U_2)$ is a bivariate normal distribution with mean 0, variance 1, and correlation $\rho$.
Define the following $U$-nodes
\begin{align}
a &= \text{region00}\\
b &= \text{region01} \setminus \text{region10}\\
c &= \text{region10} \setminus \text{region01}\\
d &= \text{region11}\\
e &= \text{region01} \cap \text{region10}.
\end{align}
We can calculate the probability distribution over the nodes under the normality assumption. The function `ex.calculate_Ftheta_entrygame` does this job. An example is given below.

In [2]:
import IncompleteDiscreteChoice.examples as ex

# Example usage
beta1 = 0.75
beta2 = 0.25
delta1 = -0.5
delta2 = -1
rho = 0.5

X1 = 1
X2 = -1
X = [X1,X2]
theta_true = [beta1, beta2, delta1, delta2, rho]
Ftheta = ex.calculate_Ftheta_entrygame(X, theta_true)
print(Ftheta)


[0.19166129 0.04242376 0.62912002 0.09455455 0.04224038]


Now let's combine this function with the one that yields the sharp lower bounds for the conditional probabilities. For this, we first represent the model by a graph.

In [3]:
import IncompleteDiscreteChoice.idclib as idc
# Define the model
Y_nodes = [(0,0), (0,1), (1,0), (1,1)]
U_nodes = ['a', 'b', 'c', 'd', 'e']
edges = [
    ('a', (0,0)),
    ('b', (0,1)),
    ('c', (1,0)),
    ('d', (1,1)),
    ('e', (0,1)),
    ('e', (1,0))
]
gmodel = idc.BipartiteGraph(Y_nodes, U_nodes, edges)

Now, for any $\theta$, we can calculates the sharp identifying restrictions (lower bounds).

In [4]:
# Use the calculate_sharp_lower_bound to calculate probabilities.
results,sharp_lower_bounds = gmodel.calculate_sharp_lower_bound(Ftheta)

# Show results
idc.print_table(results)

Subset of Y-nodes                    Exclusive U-nodes             Sharp Lower Bound  
{(0, 0)}                             {'a'}                         0.192              
{(0, 1)}                             {'b'}                         0.042              
{(1, 0)}                             {'c'}                         0.629              
{(1, 1)}                             {'d'}                         0.095              
{(0, 1), (0, 0)}                     {'b', 'a'}                    0.234              
{(1, 0), (0, 0)}                     {'c', 'a'}                    0.821              
{(1, 1), (0, 0)}                     {'d', 'a'}                    0.286              
{(0, 1), (1, 0)}                     {'b', 'c', 'e'}               0.714              
{(0, 1), (1, 1)}                     {'b', 'd'}                    0.137              
{(1, 0), (1, 1)}                     {'d', 'c'}                    0.724              
{(0, 1), (1, 0), (0, 0)}             {'b', 

Now, let's try to get the identified set. Suppose the true DGP is such that it selects (1,0) whenever multiple equilibria exist. Then, the probability allocation is
\begin{align}
P((0,0)|x)&=F_\theta(a|x)\\
P((0,1)|x)&=F_\theta(b|x)\\
P((1,0)|x)&=F_\theta(c|x)+F_\theta(e|x)\\
P((1,1)|x)&=F_\theta(d|x).
\end{align}

In [6]:
P0 = sharp_lower_bounds[1:5]
P0[2] = 1 - (P0[0] + P0[1] + P0[3])
print(P0)

[0.19166129 0.04242376 0.67136041 0.09455455]


Now, let's calculate the probabilities of all events under $P$.


In [7]:
results, subset_probabilities = idc.calculate_subset_probabilities(P0, Y_nodes)

print(f"{'Subset of Y-nodes':<45} {'P(A|x)':<20}")
print("="*65)
for subset_set, subset_prob in results:
    print(f"{str(subset_set):<45} {subset_prob:<20.3f}")

Subset of Y-nodes                             P(A|x)              
()                                            0.000               
((0, 0),)                                     0.192               
((0, 1),)                                     0.042               
((1, 0),)                                     0.671               
((1, 1),)                                     0.095               
((0, 0), (0, 1))                              0.234               
((0, 0), (1, 0))                              0.863               
((0, 0), (1, 1))                              0.286               
((0, 1), (1, 0))                              0.714               
((0, 1), (1, 1))                              0.137               
((1, 0), (1, 1))                              0.766               
((0, 0), (0, 1), (1, 0))                      0.905               
((0, 0), (0, 1), (1, 1))                      0.329               
((0, 0), (1, 0), (1, 1))                      0.958           

Compare this to the sharp lower bound. Not surprisingly, the conditional probability $P(A|x)$ is above the sharp lower bound for any event. Hence, $\theta=(0.75, 0.25, -0.5, -1, 0.5)$ is in $\Theta_I(P)$. Now, lets see if there are other parameter values that are in the sharp identified set.

For simplicity, suppose $\beta_1=0.75$, $\beta_2=0.25$, and $\rho=0.5$ is known. The following code calculates the set of $(\delta_1,\delta_2)$ that are compatible with the sharp identifying restrictions.

It is hard to plot $\Theta_I(P_0)$, but we can get its projection (to the $j$-th coordinate) by solving the following optimization problem.

\begin{align*}\max/\min_{\theta\in\Theta} &~\theta_j\\
s.t.&~P(A|x)\ge \nu_\theta(A|x),~x\in\mathcal X, A\subseteq \mathcal Y.
\end{align*}

The following code computes the projection of the sharp identification region for a component of your choice (there will be a prompt to choose the component).

In [16]:
import numpy as np
from scipy.optimize import minimize, NonlinearConstraint
from itertools import product

# Define your lower and upper bounds
lower = np.array([-2, -2, -2.5, -2.5, 0.01])
upper = np.array([2, 2, -0.01, -0.01, 0.8])

# X values
values = [-1,1]
X_values = list(product(values, values))

# Set P0(A|X)
subset_probabilities_all = []
for X in X_values:
    Ftheta = ex.calculate_Ftheta_entrygame(np.array(X), theta_true)
    _, sharp_lower_bounds = gmodel.calculate_sharp_lower_bound(Ftheta)
    P0 = sharp_lower_bounds[1:5]
    P0[2] = 1 - (P0[0] + P0[1] + P0[3])
    _, subset_probabilities = idc.calculate_subset_probabilities(P0, Y_nodes)
    subset_probabilities_all.append(subset_probabilities)

# Function to create the objective function based on the user-selected component
def create_objective_function(component_index, maximize=True):
    if maximize:
        return lambda theta: -theta[component_index]
    else:
        return lambda theta: theta[component_index]

# Define the constraint function
def constraint(theta):
    sharp_lower_bounds_all = []
    for X in X_values:
        Ftheta = ex.calculate_Ftheta_entrygame(np.array(X), theta)
        _, sharp_lower_bounds = gmodel.calculate_sharp_lower_bound(Ftheta)
        sharp_lower_bounds_all.append(sharp_lower_bounds)

    constraints = np.array(sharp_lower_bounds_all) - np.array(subset_probabilities_all)
    # Stack constraints into a single array
    return np.hstack(constraints)

# Define the bounds for the optimization
bounds = [(l, u) for l, u in zip(lower, upper)]

# Create a NonlinearConstraint object
nonlinear_constraint = NonlinearConstraint(constraint, -np.inf, 1e-4)

# Define initial guess for theta
initial_theta = np.mean([lower, upper], axis=0)

# User specifies the component of theta to optimize
component_index = int(input("Enter the component index of theta to optimize (0-based index): "))

# Run the optimization for maximization
objective_max = create_objective_function(component_index, maximize=True)
result_max = minimize(objective_max, initial_theta, method='trust-constr', bounds=bounds, constraints=[nonlinear_constraint], options={'verbose': 1})

# Run the optimization for minimization
objective_min = create_objective_function(component_index, maximize=False)
result_min = minimize(objective_min, initial_theta, method='trust-constr', bounds=bounds, constraints=[nonlinear_constraint], options={'verbose': 1})

# Check if the optimization for maximization was successful
if result_max.success:
    max_theta_value = -result_max.fun
    optimized_theta_max = result_max.x
    print(f"The maximum value of the component {component_index} of theta that satisfies the condition is: {max_theta_value}")
else:
    print("Maximization optimization failed.")
    print(result_max.message)

# Check if the optimization for minimization was successful
if result_min.success:
    min_theta_value = result_min.fun
    optimized_theta_min = result_min.x
    print(f"The minimum value of the component {component_index} of theta that satisfies the condition is: {min_theta_value}")
else:
    print("Minimization optimization failed.")
    print(result_min.message)

Enter the component index of theta to optimize (0-based index): 3
`gtol` termination condition is satisfied.
Number of iterations: 34, function evaluations: 180, CG iterations: 39, optimality: 1.00e-08, constraint violation: 0.00e+00, execution time: 0.33 s.
`gtol` termination condition is satisfied.
Number of iterations: 33, function evaluations: 168, CG iterations: 46, optimality: 7.73e-10, constraint violation: 0.00e+00, execution time: 0.31 s.
The maximum value of the component 3 of theta that satisfies the condition is: -1.000312360102826
The minimum value of the component 3 of theta that satisfies the condition is: -1.0008187443380954


For example, the projection of $\Theta_I(P_0)$ to the 3rd coordinate (i.e., $\delta_1$; use 2 (0-based index) as an input above) is $[-0.501,-0.498]$.